### Préparation de l'environnement

Ci-dessous quelques imports et précautions préalables à notre travail. Il n'est pas inutile de les parcourir.
Si nécessaire créer un bloc au démarrage pour installer toutes les librairies nécessaires en exécutant chacune leur tour les commandes suivantes:

In [314]:
# imports
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

# stabilité du notebook d'une exécution à l'autre
random=np.random.default_rng(42) 

# jolies figures directement dans le notebook
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Création des dataframes

Nous commençons par importer les CSV afin de crée les dataframes correspondant.

In [315]:
def load_data(fileName):
    csv_path = os.path.join('../data', fileName)
    df = pd.read_csv(csv_path)
    return df

In [316]:
employee_survey_data = load_data('employee_survey_data.csv')
manager_survey_data = load_data('manager_survey_data.csv')
in_time = load_data('in_time.csv')
out_time = load_data('out_time.csv')
general_data = load_data('general_data.csv')

# Choix des données

## Éthique

Dans le cas de notre modèle et de l'utilisation de nos données il est important de déterminer étiquement la conservation de certaines données ou non. 

Pour cela nous avons procédé à un brainstorming et à une lecture des recommendations de la CNIL pour conserver uniquement les données nécessaires et ne prétant pas à une possible discrimination.

Voici la liste des données que nous ne souhaitons pas conserver pour notre modèle :

- **L’âge des employés (Age) :** Nous souhaitons rester dans la plus grande neutralité possible.Effectivement, l’âge ne doit pas nous permettre de définir si une personne est plus à même de quitter l’entreprise ou non. 
<br>

- **Le genre des employés (Gender) :** Le genre est une donnée non pertinante sur les critères qui pousserai à un turnover. Cette donnée pourrait être discrimante 
<br>

- **Le statut marital (MaritalStatus) :** Cette donnée ne nous permetrai pas d’interpreter des critères cohérent concernant les Turn-over dans l’entrprise. Ce serai une surinterpretation des données fournis.

In [317]:
#remove the columns for ethic
general_dataset = general_data.copy()
general_dataset.drop("Age", axis=1, inplace=True)
general_dataset.drop("Gender", axis=1, inplace=True)
general_dataset.drop("MaritalStatus", axis=1, inplace=True)

## Logique

Il est également nécessaire de déterminer d'une manière logique les données utile à conserver ou non. 

Pour cela il suffit d'isoler dans un premier temps les colonnes de la table ***General_data***  ou le champs de valeur est égal à 1, c'est à dire que chaque données est la même.

Pour cela il suffit d'executer cette fonction afin de déterminer les champs répondant à cette condition :

In [318]:
for col in general_data.columns:
    value = general_data[col].nunique()
    value_of = general_data[col].unique()
    if value == 1:
        print(col)
        print(value_of)

EmployeeCount
[1]
Over18
['Y']
StandardHours
[8]


On constate alors que :

- **EmployeeCount :** Cette colonne correspond à la présence ou non d'un employé dans l'entreprise dans les effectifs en 2015, on constate que chaque ligne du tableau est à 1 cela signifie donc que tous les employés présent dans les données était dans l'entreprise en 2015, il n'est donc pas intéressant de conserver cette donnée.
<br>

- **Over18 :** Cette colonne indique si l'age de l'employé est supérieure ou non à 18 ans, on constate que chaque ligne du tableau est à la valeure "Y" cela signifie donc que en 2015 chaque employé était majeur, il n'est donc pas intéressant de conserver cette donnée. 
<br>

- **StandardHours :** Cette colonne spécifie le nombre d'heure inscrite sur le contrat de l'employé, chaque ligne étant positionnée à 8h on en déduit que tous les employés disposait du même type de contrat en 2015. Il n'est donc pas intéressant de conserver cette donnée.

<div class="alert alert-block alert-info">
<b>Info:</b> Cette étude à également été effectuée sur les autres tables mais aucune valeur unique est apparue, nous conserverons alors toutes les données de celles-ci.
</div>


In [319]:
# remove the non revelent columns
general_dataset.drop("Over18", axis=1, inplace=True)
general_dataset.drop("EmployeeCount", axis=1, inplace=True)
general_dataset.drop("StandardHours", axis=1, inplace=True)

## Conclusion

Après avoir selectionné les données à conserver en fonction de différents facteurs voici une liste exhaustive de celles conservées pour la réalisation de notre modèle :

###  <font color='green'> General_data </font>

- **Attrition :** L'objet de notre étude, est-ce que l'employé a quitté l'entreprise durant l'année 2016 ?
<br>

- **BusinessTravel :** A quel fréquence l'employé a été amené à se déplacer dans le cadre de son travail en 2015 ? (Non-Travel = jamais, Travel_Rarely= rarement, Travel_Frequently = fréquemment)
<br>

- **DistanceFromHome :** Distance en km entre le logement de l'employé et l'entreprise.
<br>

- **Education : Niveau d'étude :** 1=Avant College (équivalent niveau Bac), 2=College (équivalent Bac+2), 3=Bachelor (Bac+3), 4=Master (Bac+5) et 5=PhD (Thèse de doctorat).
<br>

- **EducationField :** Domaine d'étude, matière principale
<br>

- **EmployeeId :** l'identifiant d'un employé
<br>

- **JobLevel :** Niveau hiérarchique dans l'entreprise de 1 à 5
<br>

- **JobRole :** Métier dans l'entreprise
<br>

- **MonthlyIncome :** Salaire brut en roupies par mois
<br>

- **NumCompaniesWorked :** Nombre d'entreprises pour lequel le salarié a travaillé avant de rejoindre HumanForYou.
<br>

- **PercentSalaryHike :** % d'augmentation du salaire en 2015.
<br>

- **StockOptionLevel :** Niveau d'investissement en actions de l'entreprise par le salarié.
<br>

- **TotalWorkingYears :** Nombre d'années d'expérience en entreprise du salarié pour le même type de poste.
<br>

- **TrainingTimesLastYear :** Nombre de jours de formation en 2015
<br>

- **YearsAtCompany :** Ancienneté dans l'entreprise
<br>

- **YearsSinceLastPromotion :** Nombre d'années depuis la dernière augmentation individuelle
<br>

- **YearsWithCurrentManager :** Nombre d'années de collaboration sous la responsabilité du manager actuel de l'employé.

###  <font color='green'> Employee_survey_data </font>

- **L'environnement de travail :** : noté 1 ("Faible"), 2 ("Moyen"), 3 ("Élevé") ou 4 ("Très élevé") : EnvironmentSatisfaction
<br>

- **Son travail :** noté de 1 à 4 comme précédemment : JobSatisfaction
<br>

- **Son équilibre entre vie professionnelle et vie privée :** noté 1 ("Mauvais"), 2 ("Satisfaisant"), 3 ("Très satisfaisant") ou 4 ("Excellent") : WorkLifeBalance

###  <font color='green'> Manager_survey_data </font>

- **Une évaluation de son implication dans son travail :** notée 1 ('Faible'), 2 ("Moyenne"), 3 ("Importante") ou 4 ("Très importante") : JobInvolvement
<br>

- **Une évaluation de son niveau de performance annuel pour l'entreprise :** notée 1 ("Faible"), 2 ("Bon"), 3 ("Excellent") ou 4 ("Au delà des attentes") : PerformanceRating

## Encodage

### Encodage binaire de l'attribut Attrition
On convertit les données de l'attribut Attrition, les 'Yes' et 'No' deviennent respectivement des 1 et des 0.

In [320]:
general_dataset['Attrition'] = general_dataset['Attrition'].map({'Yes': 1, 'No':0}) # convertir la colonne Attrition en 0 et 1

fir_column = general_dataset.pop('Attrition') # retirer la colonne Attrition
general_dataset.insert(0 ,'Attrition', fir_column) # la remettre en première colonne

### One-Hot Encoding
Nous allons créer des variables supplémentaires pour représenter chacun des catégories.
**BusinessTravel**, **Department**, **EducationField**, **JobRole**

In [321]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False) # sparse=False pour obtenir un tableau numpy (et non une matrice creuse)
data_encoded = onehot_encoder.fit_transform(general_dataset[['BusinessTravel', 'Department', 'EducationField', 'JobRole']])
onehot_encoder.categories_ # affiche les catégories

[array(['Non-Travel', 'Travel_Frequently', 'Travel_Rarely'], dtype=object),
 array(['Human Resources', 'Research & Development', 'Sales'], dtype=object),
 array(['Human Resources', 'Life Sciences', 'Marketing', 'Medical',
        'Other', 'Technical Degree'], dtype=object),
 array(['Healthcare Representative', 'Human Resources',
        'Laboratory Technician', 'Manager', 'Manufacturing Director',
        'Research Director', 'Research Scientist', 'Sales Executive',
        'Sales Representative'], dtype=object)]

In [322]:
general_dataset

,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,JobLevel,JobRole,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,0,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Healthcare Representative,131160,1.0,11,0,1.0,6,1,0,0
1,1,Travel_Frequently,Research & Development,10,1,Life Sciences,2,1,Research Scientist,41890,0.0,23,1,6.0,3,5,1,4
2,0,Travel_Frequently,Research & Development,17,4,Other,3,4,Sales Executive,193280,1.0,15,3,5.0,2,5,0,3
3,0,Non-Travel,Research & Development,2,5,Life Sciences,4,3,Human Resources,83210,3.0,11,3,13.0,5,8,7,5
4,0,Travel_Rarely,Research & Development,10,1,Medical,5,1,Sales Executive,23420,4.0,12,2,9.0,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,0,Travel_Rarely,Research & Development,5,4,Medical,4406,1,Research Scientist,60290,3.0,17,1,10.0,5,3,0,2
4406,0,Travel_Rarely,Research & Development,2,4,Medical,4407,1,Laboratory Technician,26790,2.0,15,0,10.0,2,3,0,2
4407,0,Travel_Rarely,Research & Development,25,2,Life Sciences,4408,2,Sales Executive,37020,0.0,20,0,5.0,4,4,1,2
4408,0,Travel_Rarely,Sales,18,2,Medical,4409,1,Laboratory Technician,23980,0.0,14,1,10.0,2,9,7,8


# Fusion des tables
Nous allons fusionner les dataframes selon l'id de l'employé pour inclure les evaluations d'impliquation et les évaluations de niveaux de performance.

In [323]:
#merge the dataframes
survey_dataset = pd.merge(employee_survey_data, manager_survey_data, on='EmployeeID')
Dataset = pd.merge(general_dataset, survey_dataset, on='EmployeeID')
Dataset.head(3)

,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,JobLevel,JobRole,MonthlyIncome,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,0,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Healthcare Representative,131160,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,1,Travel_Frequently,Research & Development,10,1,Life Sciences,2,1,Research Scientist,41890,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,0,Travel_Frequently,Research & Development,17,4,Other,3,4,Sales Executive,193280,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3


# Nettoyage des données

Nous allons supprimer les valeurs manquantes.
Nous constatons qu'il y a des valeurs manquantes pour 110 employés.

In [324]:
sample_incomplete_rows = Dataset[Dataset.isnull().any(axis=1)]
len(sample_incomplete_rows.index)

110

Cela correspond à moins de 3% de nos valeurs.
Nous faisons donc le choix de supprimer les lignes contenant des valeurs manquantes.

In [325]:
(len(sample_incomplete_rows.index)/len(Dataset.index))*100

2.494331065759637

In [326]:
Dataset.dropna(inplace=True)
Dataset

,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,JobLevel,JobRole,MonthlyIncome,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,0,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Healthcare Representative,131160,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,1,Travel_Frequently,Research & Development,10,1,Life Sciences,2,1,Research Scientist,41890,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,0,Travel_Frequently,Research & Development,17,4,Other,3,4,Sales Executive,193280,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,0,Non-Travel,Research & Development,2,5,Life Sciences,4,3,Human Resources,83210,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,0,Travel_Rarely,Research & Development,10,1,Medical,5,1,Sales Executive,23420,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4404,0,Travel_Rarely,Sales,4,3,Other,4405,2,Human Resources,35390,...,6.0,2,6,1,5,3.0,4.0,3.0,2,3
4405,0,Travel_Rarely,Research & Development,5,4,Medical,4406,1,Research Scientist,60290,...,10.0,5,3,0,2,4.0,1.0,3.0,3,3
4406,0,Travel_Rarely,Research & Development,2,4,Medical,4407,1,Laboratory Technician,26790,...,10.0,2,3,0,2,4.0,4.0,3.0,2,3
4407,0,Travel_Rarely,Research & Development,25,2,Life Sciences,4408,2,Sales Executive,37020,...,5.0,4,4,1,2,1.0,3.0,3.0,3,4


expliquer les nan time

on vire les col > 50%

In [331]:
#prepare time dataset
import datetime as dt


def dropNan(dataset):

    data = dataset.copy()
    colNames = [col for col in data.columns]

    percent= [round(100-((data[col].count()/len(data.index))*100),2) for col in data.columns]
    indexes = [index for index, value in enumerate(percent) if value > 50]

    for i in indexes:
        data.drop(colNames[i], axis=1, inplace=True)
        
    data.fillna(0, inplace=True)
    
    return data

explication

In [332]:
inTime_prep = dropNan(in_time)
outTime_prep = dropNan(out_time)

In [ ]:
inTime_prep